In [1]:
!wget -O moviedataset.zip https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/moviedataset.zip
print('unziping ...')
!unzip -o -j moviedataset.zip 


--2020-03-26 13:35:40--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/moviedataset.zip
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.196
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 160301210 (153M) [application/zip]
Saving to: 'moviedataset.zip'

moviedataset.zip    100%[===================>] 152.88M  13.1MB/s    in 13s     

2020-03-26 13:35:54 (11.9 MB/s) - 'moviedataset.zip' saved [160301210/160301210]

unziping ...
Archive:  moviedataset.zip
  inflating: links.csv               
  inflating: movies.csv              
  inflating: ratings.csv             
  inflating: README.txt              
  inflating: tags.csv                


In [2]:
import pandas as pd
from math import sqrt
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
movies_df = pd.read_csv('movies.csv')
ratings_df = pd.read_csv('ratings.csv')

In [5]:
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
movies_df['year'] = movies_df.title.str.extract('(\(\d\d\d\d\))', expand =False)
movies_df['year'] = movies_df.year.str.extract('(\d\d\d\d)', expand = False)
movies_df['title'] = movies_df.title.str.replace('(\(\d\d\d\d\))','')
movies_df['title'] = movies_df['title'].apply(lambda x: x.strip())

In [7]:
movies_df.head()

,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995


In [8]:
#Dropping the genres column
movies_df = movies_df.drop('genres', 1)

In [9]:
movies_df.head()

,movieId,title,year
0,1,Toy Story,1995
1,2,Jumanji,1995
2,3,Grumpier Old Men,1995
3,4,Waiting to Exhale,1995
4,5,Father of the Bride Part II,1995


In [10]:
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,169,2.5,1204927694
1,1,2471,3.0,1204927438
2,1,48516,5.0,1204927435
3,2,2571,3.5,1436165433
4,2,109487,4.0,1436165496


In [11]:
ratings_df = ratings_df.drop('timestamp', 1)

In [13]:
ratings_df.head()

,userId,movieId,rating
0,1,169,2.5
1,1,2471,3.0
2,1,48516,5.0
3,2,2571,3.5
4,2,109487,4.0


In [14]:
"""
The process for creating a User Based recommendation system is as follows:
- Select a user with the movies the user has watched
- Based on his rating to movies, find the top X neighbours 
- Get the watched movie record of the user for each neighbour.
- Calculate a similarity score using some formula
- Recommend the items with the highest score


Let's begin by creating an input user to recommend movies to:

Notice: To add more movies, simply increase the amount of elements in the userInput. Feel free to add more in! Just be sure to write it in with capital letters and if a movie starts with a "The", like "The Matrix" then write it in like this: 'Matrix, The' ."""

'\nThe process for creating a User Based recommendation system is as follows:\n- Select a user with the movies the user has watched\n- Based on his rating to movies, find the top X neighbours \n- Get the watched movie record of the user for each neighbour.\n- Calculate a similarity score using some formula\n- Recommend the items with the highest score\n\n\nLet\'s begin by creating an input user to recommend movies to:\n\nNotice: To add more movies, simply increase the amount of elements in the userInput. Feel free to add more in! Just be sure to write it in with capital letters and if a movie starts with a "The", like "The Matrix" then write it in like this: \'Matrix, The\' .'

In [16]:
userInput = [
            {'title':'Breakfast Club, The', 'rating':5},
            {'title':'Toy Story', 'rating':3.5},
            {'title':'Jumanji', 'rating':2},
            {'title':"Pulp Fiction", 'rating':5},
            {'title':'Akira', 'rating':4.5}
]
inputMovies = pd.DataFrame(userInput)
inputMovies

,rating,title
0,5.0,"Breakfast Club, The"
1,3.5,Toy Story
2,2.0,Jumanji
3,5.0,Pulp Fiction
4,4.5,Akira


In [17]:
inputId = movies_df[movies_df['title'].isin(inputMovies['title'].tolist())]
inputMovies = pd.merge(inputId,inputMovies)
inputMovies = inputMovies.drop('year',1)
inputMovies

,movieId,title,rating
0,1,Toy Story,3.5
1,2,Jumanji,2.0
2,296,Pulp Fiction,5.0
3,1274,Akira,4.5
4,1968,"Breakfast Club, The",5.0


In [18]:
userSubset = ratings_df[ratings_df['movieId'].isin(inputMovies['movieId'].tolist())]
userSubset.head()

,userId,movieId,rating
19,4,296,4.0
441,12,1968,3.0
479,13,2,2.0
531,13,1274,5.0
681,14,296,2.0


In [19]:
userSubsetGroup = userSubset.groupby(['userId'])

In [21]:
userSubsetGroup.get_group(13)

,userId,movieId,rating
479,13,2,2.0
531,13,1274,5.0


In [22]:
userSubsetGroup = sorted(userSubsetGroup, key = lambda x: len(x[1]), reverse=True)

In [23]:
userSubsetGroup[0:3]

[(75,       userId  movieId  rating
  7507      75        1     5.0
  7508      75        2     3.5
  7540      75      296     5.0
  7633      75     1274     4.5
  7673      75     1968     5.0), (106,       userId  movieId  rating
  9083     106        1     2.5
  9084     106        2     3.0
  9115     106      296     3.5
  9198     106     1274     3.0
  9238     106     1968     3.5), (686,        userId  movieId  rating
  61336     686        1     4.0
  61337     686        2     3.0
  61377     686      296     4.0
  61478     686     1274     4.0
  61569     686     1968     5.0)]

In [24]:
userSubsetGroup = userSubsetGroup[0:100]

In [26]:
pearsonCorrelationDict = {}

for name, group in userSubsetGroup:
    group= group.sort_values(by = 'movieId')
    inputMovies = inputMovies.sort_values(by='movieId')
    nRatings = len(group)
    temp_df= inputMovies[inputMovies['movieId'].isin(group['movieId'].tolist())]
    tempRatingList = temp_df['rating'].tolist()
    tempGroupList = group['rating'].tolist()
    Sxx = sum([i**2 for i in tempRatingList]) - pow(sum(tempRatingList),2)/float(nRatings)
    Syy = sum([i**2 for i in tempGroupList]) - pow(sum(tempGroupList),2)/float(nRatings)
    Sxy = sum(i*j for i, j in zip(tempRatingList, tempGroupList)) - sum(tempRatingList)*sum(tempGroupList)/float(nRatings)
    
    if Sxx!=0 and Syy!=0:
        pearsonCorrelationDict[name] = Sxy/sqrt(Sxx*Syy)
    else:
        pearsonCorrelationDict[name] = 0

In [27]:
pearsonCorrelationDict.items()

[(4099, 0.05860090386731196),
 (10248, -0.24806946917841693),
 (1040, 0.9434563530497265),
 (2065, 0.4385290096535153),
 (17944, 0.2713848825944774),
 (17438, 0.7093169886164387),
 (12325, 0.9616783115081544),
 (5165, -0.4385290096535153),
 (11314, 0.8204126541423654),
 (11827, 0.4902903378454601),
 (8245, 0.0),
 (13366, 0.8951435925492911),
 (6207, 0.9615384615384616),
 (13888, 0.2508726030021272),
 (7235, 0.6981407669689391),
 (16456, 0.7161148740394331),
 (10315, 0.537086155529574),
 (18509, 0.1322214713369862),
 (2128, 0.5860090386731196),
 (1625, 0.716114874039432),
 (17501, 0.8168748513121271),
 (17502, 0.8272781516947562),
 (13923, 0.3516054232038718),
 (15466, 0.7205766921228921),
 (13934, 0.17200522903844556),
 (10863, 0.6020183016345595),
 (4208, 0.29417420270727607),
 (3186, 0.6784622064861935),
 (12916, 0.5860090386731196),
 (4725, -0.08006407690254357),
 (12921, 0.6611073566849309),
 (16506, 0.5481612620668942),
 (106, 0.5860090386731182),
 (10368, 0.4688072309384945),
 (1

In [28]:
pearsonDF = pd.DataFrame.from_dict(pearsonCorrelationDict, orient = 'index')
pearsonDF.columns = ['similarityIndex']
pearsonDF['userId'] = pearsonDF.index
pearsonDF.index = range(len(pearsonDF))
pearsonDF.head()

,similarityIndex,userId
0,0.058601,4099
1,-0.248069,10248
2,0.943456,1040
3,0.438529,2065
4,0.271385,17944


In [29]:
topUsers = pearsonDF.sort_values(by = 'similarityIndex', ascending=False)[0:50]
topUsers.head()

,similarityIndex,userId
6,0.961678,12325
90,0.961538,10707
12,0.961538,6207
50,0.960769,13053
2,0.943456,1040


In [30]:
topUsersRating = topUsers.merge(ratings_df, left_on = 'userId', right_on='userId', how='inner')
topUsersRating.head()

,similarityIndex,userId,movieId,rating
0,0.961678,12325,1,3.5
1,0.961678,12325,2,1.5
2,0.961678,12325,3,3.0
3,0.961678,12325,5,0.5
4,0.961678,12325,6,2.5


In [31]:
topUsersRating['weightedRating'] = topUsersRating['similarityIndex']*topUsersRating['rating']
topUsersRating.head()

,similarityIndex,userId,movieId,rating,weightedRating
0,0.961678,12325,1,3.5,3.365874
1,0.961678,12325,2,1.5,1.442517
2,0.961678,12325,3,3.0,2.885035
3,0.961678,12325,5,0.5,0.480839
4,0.961678,12325,6,2.5,2.404196


In [33]:
tempTopUsersRating = topUsersRating.groupby('movieId').sum()[['similarityIndex', 'weightedRating']]
tempTopUsersRating.columns = ['sum_similarityIndex','sum_weightedRating']
tempTopUsersRating.head()

,sum_similarityIndex,sum_weightedRating
movieId,,
1,38.376281,140.800834
2,38.376281,96.656745
3,10.253981,27.254477
4,0.929294,2.787882
5,11.723262,27.151751


In [34]:
#Creates an empty dataframe
recommendation_df = pd.DataFrame()
#Now we take the weighted average
recommendation_df['weighted average recommendation score'] = tempTopUsersRating['sum_weightedRating']/tempTopUsersRating['sum_similarityIndex']
recommendation_df['movieId'] = tempTopUsersRating.index
recommendation_df.head()

,weighted average recommendation score,movieId
movieId,,
1,3.668955,1
2,2.518658,2
3,2.657941,3
4,3.000000,4
5,2.316058,5


In [35]:
movies_df.loc[movies_df['movieId'].isin(recommendation_df.head(10)['movieId'].tolist())]

,movieId,title,year
0,1,Toy Story,1995
1,2,Jumanji,1995
2,3,Grumpier Old Men,1995
3,4,Waiting to Exhale,1995
4,5,Father of the Bride Part II,1995
5,6,Heat,1995
6,7,Sabrina,1995
7,8,Tom and Huck,1995
8,9,Sudden Death,1995
9,10,GoldenEye,1995
